# Analisi qualitativa per USERS

In [1]:
import pymongo
from pymongo import MongoClient

from pandas import DataFrame, read_csv
import json
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample

cu = db.users.find()
cs = db.shots.find()
cf = db.followers.find()
cl = db.likes.find()
ct = db.teams.find()
cutfs = db.user_team_first_shot.find()
csk = db.skills.find()

In [2]:
usernames_no_repeated = []
for user in cutfs:
    if user['user'] not in usernames_no_repeated:
        usernames_no_repeated.append(user['user'])

In [ ]:
cu[0]

In [3]:
with open('/home/simo/UnInfo/Tesi Magistrale/Dataset/features/skills_tot.json') as f:
    data = json.load(f)

In [4]:
len(data)

12850

In [5]:
data[0]

{u'skills': [u'design leadership',
  u'design strategy',
  u'engineering',
  u'interaction design',
  u'product design',
  u'user experience',
  u'user interface',
  u'visual design'],
 u'user': u'citrusbyte'}

### Skills piu frequenti

In [ ]:
skills_found = []

i=0

for user in data:
    skills = user['skills']
    for s in skills:
        found = False
        for elem in skills_found:
            if elem['skill'] == s:
                elem.update({'count': elem['count']+1})
                found = True
                break

        if not found:
            sf = {'skill': s, 'count': 1}
            skills_found.append(sf)
    i=i+1
    print i

scrivi su file

In [ ]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/skills_frequences.txt", mode='a') as f:
    for s in skills_found:
        f.write(str(s['skill'])+' - '+str(s['count'])+'\n')

leggi da file

In [6]:
skills_found = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/skills_frequences.txt", "r") as ins:
    for line in ins:
        skill = line[0:line.rfind('-')-1]
        count = line[line.rfind('-')+2:len(line)-1]
        elem = {'skill': skill, 'count': count}
        skills_found.append(elem)

n skills piu frequenti

In [7]:
def n_most_frequent(n, skill_list):
    max_skill_prev = {'count':1000000}
    for i in range(0,n):
        max = 0
        max_skill = {}
        for t in skill_list:
            if int(t['count']) > max and int(t['count']) < int(max_skill_prev['count']):
                max = int(t['count'])
                max_skill = t
                
        
        max_skill_prev = max_skill
        print max_skill

In [8]:
n_most_frequent(10,skills_found)

{'count': '3224', 'skill': 'web design'}
{'count': '3012', 'skill': 'ui'}
{'count': '2992', 'skill': 'ux'}
{'count': '2843', 'skill': 'illustration'}
{'count': '2721', 'skill': 'branding'}
{'count': '2021', 'skill': 'product design'}
{'count': '1897', 'skill': 'ui design'}
{'count': '1841', 'skill': 'graphic design'}
{'count': '1228', 'skill': 'ux design'}
{'count': '1200', 'skill': 'design'}


### User con piu skills

In [10]:
max = 0
max_user = {}
for user in data:
    if len(user['skills']) > max:
        max = len(user['skills'])
        max_user = user

max_user

{u'skills': [u'apps',
  u'branding',
  u'front-end',
  u'illustration',
  u'interaction',
  u'interaction design',
  u'mobile',
  u'motion',
  u'motion design',
  u'product design',
  u'ui',
  u'user experience',
  u'user interface',
  u'ux',
  u'web design'],
 u'user': u'hellohelloteam'}

### Frequneza degli utenti con determinate skills

In [11]:
def freq_user_with_skill(skill, users):
    users_count = 0
    for user in users:
        if skill in user['skills']:
            users_count = users_count+1
            
    f = float(users_count)/float(len(users))
    print skill +' - '+ str(f)

In [12]:
freq_user_with_skill('web design', data)

web design - 0.250894941634


In [15]:
freq_user_with_skill('ui', data)

ui - 0.23439688716


### Utente con più shots (tra gli utenti nei teams)

In [18]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [37]:
shot_number_for_user_in_team = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/shot_number_for_user_in_team.txt", "r") as ins:
    for line in ins:
        pr_occ = find_nth(line, '-', 1)
        
        n = line.count('-')
        sec_occ = find_nth(line, '-', n)
        
        team = line[0:pr_occ-1]
        user = line[pr_occ+2:sec_occ-1]
        n_shots = line[sec_occ+2:len(line)-1]
        
        elem = {'team': team, 'user': user, 'n_shots': n_shots}
        shot_number_for_user_in_team.append(elem)

In [38]:
max_user = ""
max_shot = 0

for u in shot_number_for_user_in_team:
    if float(u['n_shots']) > max_shot:
        max_shot = float(u['n_shots'])
        max_user = u['user']

print max_user, max_shot

fraserdavidson 1254.0


In [27]:
# metodo vecchio altrernativo

'''max = 0
max_user = {}

i=0

for u in usernames_no_repeated:
    active_user = db.shots.find({'username': u})
    if active_user.count() > 0:
        if len(active_user[0]['shots']) > max:
            max = len(active_user[0]['shots'])
            max_user = active_user[0]
    
    i=i+1
    print i
    
print 'number of shots: ' + str(max)
print max_user'''
    

"max = 0\nmax_user = {}\n\ni=0\n\nfor u in usernames_no_repeated:\n    active_user = db.shots.find({'username': u})\n    if active_user.count() > 0:\n        if len(active_user[0]['shots']) > max:\n            max = len(active_user[0]['shots'])\n            max_user = active_user[0]\n    \n    i=i+1\n    print i\n    \nprint 'number of shots: ' + str(max)\nprint max_user"

### Skill dello User con più shots

In [41]:
for user in data:
    if user['user'] == max_user:
        print user['user']
        print user['skills']

fraserdavidson
[u'after effects', u'branding', u'cinema 4d', u'icon design', u'illustration', u'illustrator', u'logo', u'logo design', u'photoshop']


### Tags più frequenti negli shots dello user più attivo

In [42]:
user = db.shots.find({'username': max_user})[0]['shots']
len(user['shots'])

1254

In [44]:
tags_found_in_max_user = []

for s in db.shots.find({'username': max_user})[0]['shots']:
    tags = s['tags']
    for t in tags:
        found = False
        for elem in tags_found_in_max_user:
            if elem['tag'] == t:
                elem.update({'count': elem['count']+1})
                found = True
                break

        if not found:
            tf = {'tag': t, 'count': 1}
            tags_found_in_max_user.append(tf)
    

In [45]:
len(tags_found_in_max_user)

1162

In [46]:
def n_most_frequent(n, tag_list):
    max_tag_prev = {'count':1000000}
    for i in range(0,n):
        max = 0
        max_tag = {}
        for t in tag_list:
            if int(t['count']) > max and int(t['count']) < int(max_tag_prev['count']):
                max = int(t['count'])
                max_tag = t
                
        
        max_tag_prev = max_tag
        print max_tag

In [47]:
n_most_frequent(10, tags_found_in_max_user)

{'count': 295, 'tag': u'logo'}
{'count': 259, 'tag': u'football'}
{'count': 212, 'tag': u'animated'}
{'count': 211, 'tag': u'gif'}
{'count': 190, 'tag': u'animation'}
{'count': 119, 'tag': u'sports'}
{'count': 80, 'tag': u'rugby'}
{'count': 68, 'tag': u'nfl'}
{'count': 63, 'tag': u'logos'}
{'count': 48, 'tag': u'sport'}
